## Importing Required Libraries

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import tensorflow as tf

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Input, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from keras.layers import MaxPooling2D, Flatten, Conv2D, Softmax
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras_preprocessing import image

## Importing Dataset and Extracting Data

In [7]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Labe6jnY_iXYhLQooSDNXqg3cTD5MNLZwWxDcnfMrf47',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'classificationofarrhythmia-donotdelete-pr-5b1yzw9anvyg4c'
object_key = 'IBM Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [8]:
from io import BytesIO
import zipfile

unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
ls

data/


In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
x_train = train_datagen.flow_from_directory(directory=r'data/train',
                                            target_size=(128,128),
                                            batch_size=32,
                                            class_mode='categorical',
                                            classes=['Left Bundle Branch Block', 'Normal',
                                                     'Premature Atrial Contraction', 
                                                     'Premature Ventricular Contractions',
                                                     'Right Bundle Branch Block', 'Ventricular Fibrillation'],
                                            )

x_test = test_datagen.flow_from_directory(directory=r'data/test',
                                          target_size=(128,128),
                                          batch_size=32,
                                          class_mode='categorical',
                                          classes=['Left Bundle Branch Block', 'Normal',
                                                     'Premature Atrial Contraction', 
                                                     'Premature Ventricular Contractions',
                                                     'Right Bundle Branch Block', 'Ventricular Fibrillation'],
                                          )

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


## CNN Model Building

In [15]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(35))
model.add(Dense(6, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dropout (Dropout)           (None, 32768)             0

## CNN Model Training

In [16]:
model_history = model.fit(x_train,
                          steps_per_epoch=len(x_train),
                          epochs=25,
                          validation_data=x_test,
                          validation_steps=len(x_test),)

Epoch 1/25
480/480 [==============================] - 347s 721ms/step - loss: 0.9212 - accuracy: 0.6909 - val_loss: 0.6773 - val_accuracy: 0.7546
Epoch 2/25
480/480 [==============================] - 344s 717ms/step - loss: 0.4193 - accuracy: 0.8663 - val_loss: 0.5708 - val_accuracy: 0.8126
Epoch 3/25
480/480 [==============================] - 344s 716ms/step - loss: 0.3670 - accuracy: 0.8848 - val_loss: 0.5142 - val_accuracy: 0.8293
Epoch 4/25
480/480 [==============================] - 342s 713ms/step - loss: 0.3803 - accuracy: 0.8846 - val_loss: 0.4707 - val_accuracy: 0.8353
Epoch 5/25
480/480 [==============================] - 342s 714ms/step - loss: 0.3444 - accuracy: 0.8958 - val_loss: 0.5059 - val_accuracy: 0.8309
Epoch 6/25
480/480 [==============================] - 343s 715ms/step - loss: 0.3058 - accuracy: 0.9073 - val_loss: 0.4740 - val_accuracy: 0.8527
Epoch 7/25
480/480 [==============================] - 346s 720ms/step - loss: 0.2687 - accuracy: 0.9189 - val_loss: 0.3944 -

In [17]:
model.save('ECG.h5')

In [18]:
#compress the file
!tar -zcvf arrhythmia-classification-model.tgz ECG.h5

ECG.h5


## IBM Deployment

In [19]:
!pip install ibm_watson_machine_learning

In [20]:
from ibm_watson_machine_learning import APIClient

In [21]:
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "ChIkpkbcs--Hrju5LZSVzaEK-4HxxXYXSUez9G6VNuP6"
}

In [22]:
client = APIClient(wml_credentials)

In [23]:
client

In [24]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -------------------------  ------------------------
ID                                    NAME                       CREATED
11f2d9ff-4100-4f35-a236-3cc9706e0a79  arrhythmia_classification  2022-11-19T20:37:17.381Z
------------------------------------  -------------------------  ------------------------


In [25]:
space_id = "11f2d9ff-4100-4f35-a236-3cc9706e0a79"

In [26]:
client.set.default_space(space_id)

'SUCCESS'

In [27]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [28]:
software_space_uid = client.software_specifications.get_id_by_name("tensorflow_rt22.1-py3.9")

In [29]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [30]:
model_details = client.repository.store_model(model="arrhythmia-classification-model.tgz", meta_props={
    client.repository.ModelMetaNames.NAME:"cnn model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [31]:
model_id = client.repository.get_model_id(model_details)

In [32]:
model_id

'2f7b57cc-ba9e-49a8-b851-23b7290c18b3'